In [1]:
import yfinance as yf
import pandas as pd
from openpyxl import load_workbook
import warnings
import os
warnings.filterwarnings('ignore')

In [2]:
# Code to the main function. It extracts the correct tickers values, even if passed as tuples, lists and also removes incorrect values.

def get_last_dividend(ticker, *args) -> pd.DataFrame:
  '''
  Params:
  ticker = Ticker or list of tickers whose stock you want information.
  ============================================================================
  Output:
  last_dividends = Returns a DataFrame with last dividend information for each passed ticker.
  '''
  tickers = [ticker, *args]
  last_dividends = pd.DataFrame()
  formated_tickers = []
  for ticker in tickers:
    formated_tickers.append(ticker) if isinstance(ticker, str) else None
    if isinstance(ticker, (list, tuple)):
      for n in ticker:
        formated_tickers.append(n) if isinstance(n, str) else None
  if len(formated_tickers) == 0: raise ValueError("No valid ticker was identified.")
  for form_ticker in formated_tickers:
    try:
      ticker_obj = yf.Ticker(form_ticker)
      dividends = ticker_obj.dividends.reset_index().tail(1)
      dividends['Date'] = dividends["Date"].astype(str).apply(lambda x: x.split(' ').pop(0))
      dividends['Ticker'] = ticker_obj.ticker
      last_dividends = pd.concat([last_dividends, dividends], ignore_index=True, axis=0)
    except:
      print(f'The value: {form_ticker} is not a valid ticker.')
  return last_dividends

In [3]:
# This cell shows the pandas DF output of the function.

get_last_dividend(("XPML11.SA"), ["KNIP11.SA", 2], ("PETR4.SA", "PETR4", 5))

$PETR4: possibly delisted; no timezone found


The value: PETR4 is not a valid ticker.


,Date,Dividends,Ticker
0,2025-02-19,0.920000,XPML11.SA
1,2025-02-03,0.950000,KNIP11.SA
2,2024-12-26,1.339555,PETR4.SA


In [4]:
# This cell saves the DataFrame as an excel file if it doesn't exists.

dvds = get_last_dividend(("XPML11.SA"), ["KNIP11.SA", 2], ("PETR4.SA", "PETR4", 5))
if not os.path.exists("./data/teste.xlsx"):
    os.mkdir("./data/")
    dvds.to_excel('./data/teste.xlsx', index=False)

$PETR4: possibly delisted; no timezone found


The value: PETR4 is not a valid ticker.


In [5]:
# This cell appends new values to the excel files.

workbook = load_workbook('data/teste.xlsx')  # Replace with your actual file name
sheet = workbook.active  # Assuming you want to append to the active sheet
# Assuming 'dvds' is your DataFrame with the last dividends
dvds = get_last_dividend(("XPLG11.SA"))
for index, row in dvds.iterrows():
    sheet.append([row['Date'], row['Dividends'], row['Ticker']])  # Adjust column names as needed
workbook.save('data/teste.xlsx')  # Save the changes back to the file

In [6]:
# Checking if new values were saved

pd.read_excel("./data/teste.xlsx")

,Date,Dividends,Ticker
0,2025-02-19,0.920000,XPML11.SA
1,2025-02-03,0.950000,KNIP11.SA
2,2024-12-26,1.339555,PETR4.SA
3,2025-02-03,0.820000,XPLG11.SA
